In [ ]:
%pylab inline
import numpy as np
import os
import pickle
import pycuber
import random

import gym
from gym import spaces
from gym.utils import seeding

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from torch.distributions import Categorical

from collections import namedtuple
from itertools import count
from tensorboardX import SummaryWriter

# plt.style.use('fivethirtyeight')

Так выглядит его описание и вот так можно его преобразовать.

In [ ]:
cube = pycuber.Cube()
print(str(cube))
print('')

# bad
print(str(cube).replace(' ', '').replace('\n', '').replace('[', '').replace(']', ''))

Но лучше по очереди каждую стороны.

In [ ]:
raw_state = str(cube).replace(' ', '').replace('\n', '').replace('[', '').replace(']', '')

# better
state = (
    raw_state[:9] +
    raw_state[9:12] + raw_state[21:24] + raw_state[33:36] +
    raw_state[12:15] + raw_state[24:27] + raw_state[36:39] +
    raw_state[15:18] + raw_state[27:30] + raw_state[39:42] +
    raw_state[18:21] + raw_state[30:33] + raw_state[42:45] +
    raw_state[-9:]
)

Выучим один раз one-hot для этого кубика.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [ ]:
ohe = OneHotEncoder()
le = LabelEncoder()

In [ ]:
ohe.fit(np.array(list(state)).reshape((-1, 1)))

Функция для извлечения состояния, описывающего кубик.

In [ ]:
def extract_state(cube):
    raw_state = str(cube).replace(' ', '').replace('\n', '').replace('[', '').replace(']', '')

    state = (
        raw_state[:9] +
        raw_state[9:12] + raw_state[21:24] + raw_state[33:36] +
        raw_state[12:15] + raw_state[24:27] + raw_state[36:39] +
        raw_state[15:18] + raw_state[27:30] + raw_state[39:42] +
        raw_state[18:21] + raw_state[30:33] + raw_state[42:45] +
        raw_state[-9:])
    
    return np.array(ohe.transform(np.array(list(state)).reshape((-1, 1))).todense(), dtype=np.float32).ravel()

Список возможных действий.

In [ ]:
actions = ["L", "L'", "R", "R'", "U", "U'", "D", "D'", "F", "F'", "B", "B'"]

In [ ]:
n_action = len(actions)
n_space = len(extract_state(cube))

In [ ]:
final_state = extract_state(pycuber.Cube())

In [ ]:
cube = pycuber.Cube()

In [ ]:
# for action in np.random.randint(0, n_action, 10):
#     cube.perform_step(pycuber.Step(actions[action]))
#     state = extract_state(cube)
#     print np.sum(final_state == state)

Среда в формате gym.

In [ ]:
class CubeEnv(gym.Env):
    def __init__(self, min_steps, max_steps):
        self.action_space = spaces.Discrete(n_action)
        self.observation_space = spaces.Discrete(n_space)    
        self._cube = None
        self._min_steps = min_steps
        self._max_steps = max_steps
        self._quality = 0
        
        self.seed()
        self.reset()

    def seed(self, seed=None):
        np.random.seed(seed)
        return seed

    def step(self, action):
        self._cube.perform_step(pycuber.Step(actions[action]))
        observation = extract_state(self._cube)
        reward = self.calc_reward(observation)

        return observation, reward, all(observation == final_state), {}
    
    def calc_reward(self, observation):
#         global steps_done
#         alpha = min(max(0, (steps_done - 10000) / REWARD_CHANGE_STEPS), 1)
#         new_quality = (
#             (1 - alpha) * np.mean(observation == final_state) +
#             alpha * int(all(observation == final_state))) * 100
        new_quality = np.mean(observation == final_state) * 100
        
        reward = new_quality - self._quality
        self._quality = new_quality
        return reward

    def reset(self):
        self._cube = pycuber.Cube()
        random_actions = np.random.randint(
            0, n_action,
            size=np.random.randint(self._min_steps, self._max_steps + 1))
        self._cube.perform_algo(pycuber.Formula(list(map(lambda x: actions[x], random_actions))))
        observation = extract_state(self._cube)
        self.calc_reward(observation)
        
        return observation

In [ ]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_space, 150)
        self.layer2 = nn.Linear(150, 100)
        self.layer3 = nn.Linear(100, 50)
        self.head = nn.Linear(50, n_action)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        return self.head(x)

In [ ]:
def select_action(state):

#     epsilon greedy
#     global steps_done
#     sample = random.random()
#     eps_threshold = EPS_END + (EPS_START - EPS_END) * \
#         np.exp(-1. * steps_done / EPS_DECAY)
    
#     output = policy_net(state)
#     probs = np.exp((output[0].detach().numpy() + EPS) / TAU)
#     probs /= probs.sum()
    
#     if sample > eps_threshold:
#         with torch.no_grad():
#             return output.max(1)[1].view(1, 1), probs
#     else:
#         return torch.tensor([[random.randrange(n_action)]], dtype=torch.long), probs
    
    # boltzman
    categorical = Categorical(logits=policy_net(state).detach())
    
#     probs = np.exp((policy_net(state)[0].detach().numpy() + EPS) / TAU)
#     probs /= probs.sum()
#     action = np.random.choice(np.arange(n_action), p=probs)
    
#     return torch.tensor([[action]], dtype=torch.long), probs
    return categorical.sample(), categorical

In [ ]:
def test_agent(env, policy_net, n_episodes, max_episodes_length):
    test_reward = []
    test_entropy = []
    n_success_episodes = 0
    
    for i_episode in range(n_episodes):
        episode_reward = 0
        state = torch.tensor([env.reset()])
        
        for t in range(max_episodes_length):
            output = policy_net(state).detach()
            action = output.max(1)[1]
#             probs = output.exp()[0].detach().numpy() + EPS
#             probs /= probs.sum()
#             test_entropy.append(np.sum(- probs * np.log(probs + EPS)))
            categorical = Categorical(logits=output)
            test_entropy.append(float(categorical.entropy()))

            state, reward, done, _ = env.step(action.item())
            state = torch.tensor([state])
            episode_reward += reward
            if done:
                n_success_episodes += 1
                break

        test_reward.append(episode_reward)
    
    return float(n_success_episodes) / n_episodes, np.mean(test_reward), np.mean(test_entropy)

In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), dtype=torch.uint8)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken
    state_action_values = policy_net(state_batch).gather(1, action_batch.view(-1, 1))

    # compute V(s_{t+1}) for all next states.
    next_state_values = torch.zeros(BATCH_SIZE)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    
    # compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    loss = F.mse_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()
    
    weights = policy_net.state_dict()
    weights_norm = np.mean([np.mean(w.numpy() ** 2) for _, w in weights.items()])
    writer.add_scalar('network/norm_weights', weights_norm, steps_done)

In [ ]:
def train(learn_env, test_env, num_episodes):
    global steps_done
    global episode_durations
    
    for i_episode in range(num_episodes):
        state = torch.tensor([learn_env.reset()])

        for t in count():
            action, _ = select_action(state)
            next_state, reward, done, _ = learn_env.step(action.item())
            next_state = torch.tensor([next_state])
            reward = torch.tensor([reward], dtype=torch.float)

            if done:
                next_state = None

            memory.push(state, action, next_state, reward)
            state = next_state

            if done or t == max_train_episode_length - 1:
                episode_durations.append(t + 1)
                break

        optimize_model()
        if steps_done % TARGET_UPDATE == 0:
            target_net.load_state_dict(policy_net.state_dict())

        if steps_done % TEST_PERIOD == 0:
            success_rate, avg_reward, avg_entropy = test_agent(test_env, policy_net, n_episodes=100, 
                                                               max_episodes_length=100)
            print(steps_done, success_rate, avg_reward)
            
            writer.add_scalar('cube/success_rate', success_rate, steps_done)
            writer.add_scalar('cube/avg_reward', avg_reward, steps_done)
            writer.add_scalar('policy/avg_entropy', avg_entropy, steps_done)
#             pickle.dump(policy_net.state_dict(), open('pytorch/weights_{}.pkl'.format(steps_done), 'wb'))
            
        steps_done += 1

In [ ]:
EPS = 1e-100
BATCH_SIZE = 128
GAMMA = 0.9
TAU = 1.0

TARGET_UPDATE = 500
TEST_PERIOD = 100

# EPS_START = 0.9
# EPS_END = 0.05
# EPS_DECAY = 500

In [ ]:
policy_net = DQN()
target_net = DQN()
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters())
memory = ReplayMemory(1000000)
steps_done = 0

writer = SummaryWriter(log_dir='logs/random_learn_5fix-5-10random_test_10')

In [ ]:
learn_env = CubeEnv(min_steps=5, max_steps=5)
test_env = CubeEnv(min_steps=10, max_steps=10)

episode_durations = []
max_train_episode_length = 50

train(learn_env=learn_env, test_env=test_env, num_episodes=100000)

In [ ]:
learn_env = CubeEnv(min_steps=5, max_steps=10)
test_env = CubeEnv(min_steps=10, max_steps=10)

episode_durations = []
max_train_episode_length = 50

train(learn_env=learn_env, test_env=test_env, num_episodes=100000)

In [ ]:
success_rate, avg_reward, avg_entropy = \
    test_agent(test_env, policy_net, n_episodes=1000, max_episodes_length=100)

In [ ]:
success_rate